<a href="https://colab.research.google.com/github/whsu00/project/blob/with_gpu/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#@title Setup
#@markdown Run me to clone the repo, install dependencies, and set up MuJoCo!

!rm -rf * .config
!git clone -b with_gpu https://github.com/whsu00/project
!apt-get install -y libglu1-mesa-dev libgl1-mesa-dev libosmesa6-dev xvfb ffmpeg curl patchelf libglfw3 libglfw3-dev cmake zlib1g zlib1g-dev swig libosmesa6-dev libgl1-mesa-glx libglfw3
!pip install mujoco_py==2.0.2.8 mpi4py

!wget https://whsu00.com/mujoco.zip
!unzip mujoco.zip
!rm mujoco.zip
!mv .mujoco ~

MJC_PATH = "/root/.mujoco"
import os
os.environ['LD_LIBRARY_PATH'] += ':{}/mujoco200/bin'.format(MJC_PATH)
os.environ['MUJOCO_PY_MUJOCO_PATH'] = '{}/mujoco200'.format(MJC_PATH)
os.environ['MUJOCO_PY_MJKEY_PATH'] = '{}/mjkey.txt'.format(MJC_PATH)

!cp $MJC_PATH/mujoco200/bin/*.so /usr/lib/x86_64-linux-gnu/
%cd project

Cloning into 'project'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 202 (delta 38), reused 61 (delta 35), pack-reused 131
Receiving objects: 100% (202/202), 744.95 KiB | 6.77 MiB/s, done.
Resolving deltas: 100% (83/83), done.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libglu1-mesa-dev is already the newest version (9.0.0-2.1build1).
zlib1g is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
libglfw3 is already the newest version (3.2.1-1).
libglfw3-dev is already the newest version (3.2.1-1).
patchelf is already the newest version (0.9-1).
swig is already the newest version (3.0.12-1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
curl is already the newest version (7.58.0-2ubuntu3.10).
libgl1-mesa-dev is already the newest version (20.0.8-0ub

In [19]:
%load_ext autoreload
%autoreload 2
import utils, network, buffer, valor
import sys
sys.path.insert(0, './utils/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
#@title Arguments

"""
from utils.run_utils import setup_logger_kwargs
logger_kwargs = setup_logger_kwargs(exp_name, seed)

valor(lambda: gym.make(env), actor_critic=ActorCritic, ac_kwargs=dict(hidden_dims=[hid]*l),
    disc=Discriminator, dc_kwargs=dict(hidden_dims=hid),
    gamma=gamma, seed=seed, episodes_per_epoch=episodes, epochs=epochs, logger_kwargs=logger_kwargs, con_dim=con)
""";

class Args:
    env = "HalfCheetah-v2" #@param {type:"string"}
    hid = 64 #@param {type:"integer"}
    l = 2 #@param {type:"integer"}
    exp_name = "valor" #@param {type:"string"}
    seed = 0 #@param {type:"integer"}
    episodes_per_epoch = 40 #@param {type:"integer"}
    epochs = 50 #@param {type:"integer"}
    gamma = 0.99 #@param {type:"number"}
    pi_lr = 3e-4 #@param {type:"number"}
    vf_lr = 1e-3 #@param {type:"number"}
    dc_lr = 2e-3 #@param {type:"number"}
    train_v_iters = 80 #@param {type:"integer"}
    train_dc_iters = 50 #@param {type:"integer"}
    train_dc_interv = 2 #@param {type:"integer"}
    lam = 0.97 #@param {type:"number"}
    max_ep_len = 1000 #@param {type:"integer"}
    context_dim = 4 #@param {type:"integer"}
    max_context_dim = 64 #@param {type:"integer"}
    save_freq = 10 #@param {type:"integer"}
    k = 1 #@param {type:"integer"}

      
    env_fn = lambda: gym.make(env)
    actor_critic = network.ActorCritic
    ac_kwargs = {
        
    }
    disc = network.Discriminator
    dc_kwargs = {
        
    }


    


In [ ]:
from run_utils import setup_logger_kwargs
args = Args()
args.logger_kwargs = setup_logger_kwargs(args.exp_name, args.seed)

In [ ]:
#@title Run
#@markdown Uses the arguments to run `valor.py`
!python valor.py --env $env --hid $hid --l $l --gamma $gamma --seed $seed --episodes $episodes --epochs $epochs --exp_name $exp_name --con $con